In [1]:
!pip install Sastrawi


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

In [3]:
df = pd.read_excel("data/hasilCleaning.xlsx")
df

,Message
0,NaN
1,siap siap buat isi full tank bro
2,promo tebar undian apa masih bisa ikutan
3,okaaay siaaapp noted
4,buat pengguna baru aja gak sih
...,...
7064,wowww ada promo spesial akhir tahun
7065,kalo gue yang pake bakal keren gak
7066,min abis tukar nanti ambil dimana hadiahnya
7067,loh iya toh besok tahun baruan


In [4]:
df = df.dropna()

In [6]:
df.shape

(6949, 1)

## Filter Tokens

In [7]:
# Definisi fungsi filter_tokens_by_length
def filter_tokens_by_length(dataframe, column, min_words, max_words):
    # Tokenisasi kata
    words_count = dataframe[column].astype(str).apply(lambda x: len(x.split()))
    # Membuat filter untuk jumlah kata
    mask = (words_count >= min_words) & (words_count <= max_words)
    # Mengaplikasikan filter ke DataFrame
    filtered_df = dataframe[mask]
    return filtered_df

# Menggunakan filter_tokens_by_length untuk mendapatkan baris dengan jumlah kata antara 3 dan 50
min_words = 3
max_words = 50
df = filter_tokens_by_length(df, 'Message', min_words, max_words)

In [15]:
df

,Message
1,siap siap buat isi full tank bro
2,promo tebar undian apa masih bisa ikutan
3,oke siap noted
4,buat pengguna baru aja tidak sih
5,ya ayo ayo
...,...
7064,wow ada promo spesial akhir tahun
7065,kalo gue yang pake bakal keren gak
7066,min abis tukar nanti ambil dimana hadiahnya
7067,loh iya toh besok tahun baruan


## Normalisasi

In [12]:
# Normalisasi
norm = {' mikir2 ':' mikir ', ' siaaapp ':' siap ', 'okaaay ':'oke ', 'udh ':'sudah ','ga ':'tidak ',' gaskeun':' ayo ', 'wowww ':'wow ', ' haaayyuukkk ':' ayo ', ' yg ':' yang ', ' udh ':' udah ', 'wkwk ':' ', ' min ':' kak ', ' malem ':' malam', ' malem2 ':' malam ', ' sm ':' sama ', ' dy ':' dia ', ' lg ':' lagi ', ' skrg ':' sekarang ', ' ddpn ':' didepan ', ' makasi ':' makasih ', ' pertamaz ':' pertamax ', ' jg ':' juga ', ' donk ':' dong ', ' ikutann ':' ikutan ', ' banyakk ':' banyak ', ' twt ':' tweet', 'mantaap ':'mantap ', ' juarak':' juara ', 'daridulu ':'dari dulu ', 'siapp ':'siap ', ' gamau ':' tidak mau ', ' sll ':' selalu ', ' qu ':' aku ', ' krn ':' karena ', ' irii':' iri', ' muluu ':' terus ', 'mada ':'masa ', 'jgn ':'jangan ', ' jgn ':' jangan ', ' muluuu ':' terus ', 'ntar ':'nanti ', ' awtnya':' awetnya', 'gg ':'keren ', ' kerennn':' keren ', ' bisaa ':' bisa ', 'gaaa':'tidak ', " yg ": " yang ", ' nyampe':' sampai', ' nyampe ':' sampai ', ' lu ':' kamu ', ' ikhlaaasss ':' ikhlas ', ' gak ':' tidak ', ' klo ':' kalo ', ' amp ': ' sampai ', ' ga ':' tidak ', ' yaaaa':' ya ', 'betolll ':'betul ', ' kaga ':' tidak ', ' idk ':' tidak tahu ', ' jkt ':' jakarta ', ' lo ':' kamu ', ' bjir ':' ', ' kek ':' seperti ', ' yg ':' yang ', ' utk ':' untuk ', 'kismin ':'miskin ', ' kismin ':' miskin ', ' pd ':' pada ', ' dgn ':' dengan ', ' ituu ':' itu ', ' jg ':' juga ', 'yoi':'iya ', ' yoi ':' iya ', 'org2 ':'orang ', ' tak ':' tidak ', ' kyk ':' seperti ', ' sbg ':' sebagai ', ' anjjjj ':' ', ' bgt ':' banget ', 'km ':'kamu ', ' km ':' kamu', ' byk ':' banyak ', ' lg ':' lagi ', ' mrk ':' mereka ', ' blm ':' belum '}

def normalisasi(str_text):
  for i in norm:
    str_text = str_text.replace(i, norm[i])
  return str_text

df['Message'] = df['Message'].apply(lambda x: normalisasi(x))
df

C:\Users\MuhammadAulia\AppData\Local\Temp\ipykernel_25320\24246911.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Message'] = df['Message'].apply(lambda x: normalisasi(x))


,Message
1,siap siap buat isi full tank bro
2,promo tebar undian apa masih bisa ikutan
3,oke siap noted
4,buat pengguna baru aja tidak sih
5,ya ayo ayo
...,...
7064,wow ada promo spesial akhir tahun
7065,kalo gue yang pake bakal keren gak
7066,min abis tukar nanti ambil dimana hadiahnya
7067,loh iya toh besok tahun baruan


## Stopwords

In [8]:
# Stopwords
import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
more_stop_words = []

stop_words = StopWordRemoverFactory().get_stop_words()
stop_words.extend(more_stop_words)

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text

df['cleaned_text'] = df['cleaned_text'].apply(lambda x: stopword(x))
df.head()

C:\Users\MuhammadAulia\AppData\Local\Temp\ipykernel_35652\2678442150.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_text'] = df['cleaned_text'].apply(lambda x: stopword(x))


,cleaned_text
0,body ev dibuat bauksit juga dimiliki indonesia...
1,baterai merupakan komponen utama ev indonesia ...
2,indonesia memiliki semua komponen bahan dasar ...
3,ev merupakan tongkat indonesia menjadi dominas...
4,hadirnya spklu juga menjadi perhatian tamutamu...


## Tokenize

In [13]:
# Tokenize
tokenized = df['content'].apply(lambda x:x.split())
tokenized

,content
5,"[ongkir, mahal, pajak, aplikasi, gede, biaya, ..."
8,"[aplikasi, jelas, sering, banget, gofood, udah..."
9,"[pas, butuh, malah, setting, lokasi, udah, pak..."
10,"[update, aplikasinya, malah, menyusahkan, pesa..."
11,"[benar2, membantu, cuma, kadang2, apknya, suka..."
...,...
116995,"[sangat, bagus, dipahamidan, membantu]"
116996,"[gercep, kebanyakan, driver, baca, map, jadi, ..."
116997,"[sangat, membantu, promonya, banyak, sukses, s..."
116998,"[makin, kesini, malah, makin, kesana, ongkirny..."


## Stemming

In [14]:
# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(text_cleaning):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in text_cleaning:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  print(d_clean)
  return d_clean

tokenized = tokenized.apply(stemming)

tokenized.to_csv("/content/drive/MyDrive/Colab /JOKI/AnalisisSentimen(GOJEK)/data/hasil_preprocessingData.csv", index=False)

Streaming output truncated to the last 5000 lines.
cs guna ai ada solusi sama sekali ngawurrrr
pas hujan kalau emng nerima order yaudah awal gausah terima lain dr awal terima suruh nunggu ampe jam lebih malah ujunh2an cancel
bagaimana cara kirim pesan mitra kok bisa
bagus s tp tiap hari raya deket hr besar satu kalo ujan dikit ajah tidak online drivernya kecewa dikit c benar karna aku hampir tiap hari pake aplikasi gojek
sangat bantu saat ada kendara kasih bintangkarna puas layan nya tahan
baik sistem gofood nya tunggu lama tidak driver susah batal baik cari driver lebih sembari tunggu restoran siap terimakasih
semua jadi praktis
driver nya ramah baik hati
nunggu pesan gofood hampir 1 jam cancel sendiri krna ada driver udah 2 kali sama hadeh
sangat bantu banget mo buru2 pulang karna hujan jauh dr angkot gojek siaga selalu good
sangat bantu jalan
bantu banget cuma sayang kalo ujan drver nya jauh
susah banget buat tarik tunai bca jebak tol saldo gopay bisa tarik rekening bca buat top up 

KeyboardInterrupt: 

In [15]:
tokenized

,content
5,"[ongkir, mahal, pajak, aplikasi, gede, biaya, ..."
8,"[aplikasi, jelas, sering, banget, gofood, udah..."
9,"[pas, butuh, malah, setting, lokasi, udah, pak..."
10,"[update, aplikasinya, malah, menyusahkan, pesa..."
11,"[benar2, membantu, cuma, kadang2, apknya, suka..."
...,...
116995,"[sangat, bagus, dipahamidan, membantu]"
116996,"[gercep, kebanyakan, driver, baca, map, jadi, ..."
116997,"[sangat, membantu, promonya, banyak, sukses, s..."
116998,"[makin, kesini, malah, makin, kesana, ongkirny..."
